# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup
#### Load the API key and relevant Python libraries.

In [0]:
!pip install -Uq langchain tiktoken

In [0]:
import openai
import os

#from langchain import OpenAI, LLMChain
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
import tiktoken

In [0]:
with open("../azure_api-key", 'r') as f:
    secret_key = f.read()

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = secret_key

openai_model_name = "text-davinci-003"
openai_deploy_name = "model-text-davinci-003"

In [0]:
llm = AzureOpenAI(deployment_name=openai_deploy_name,
                  model_name=openai_model_name,
                  openai_api_key=openai.api_key,
                  openai_api_version=openai.api_version,
                  #prompt=prompt,
                  #max_tokens=OPENAI_MAX_TOKENS[openai_model_name],
                  temperature=0,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0,
                  best_of=1,
                  #stop=None,
                  )

## Product review text

In [0]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [0]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

In [0]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

## Identify types of emotions

In [0]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

## Identify anger

In [0]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

## Extract product and company name from customer reviews

In [0]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

## Doing multiple tasks at once

In [0]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""

response = llm(prompt)
print(response)

## Inferring topics

In [0]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

##### We have chosen to infer 5 topics

In [0]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas 

Text sample: '''{story}'''
"""

response = llm(prompt)
print(response)
print(response.strip("[]\n").split(sep=','))

##### Guiding the topic extraction:

In [0]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [0]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""

response = llm(prompt)
print(response)

In [0]:
l = eval(response)

for i in range(len(topic_list)):
    if l[i] == 1:
        print(f"ALERT: New {topic_list[i]} story!")
    else:
        print(f"No story about '{topic_list[i]}'")